In [1]:
import subprocess,os
import re
import pandas as pd
import numpy as np


In [ ]:
###获取每个transcript的region，即bed文件
bed_file <- read.table('/disk1/wenqing/tmp_data/SUPPA2/ref/gencode_hg19_transcript.bed')
head(bed_file)
all_as_types=c('A3','A5','AF','AL','MX','SE')
for(type in all_as_types){
    as_events <- read.table(paste0('/disk1/wenqing/tmp_data/SUPPA2/ref/',type,'_as_events.txt'))[,1]
    total_events <- read.table(paste0('/disk1/wenqing/tmp_data/SUPPA2/ref/',type,'_total_events.txt'))[,1]
    as_events_bed <- bed_file[bed_file[,5] %in% as_events,]
    total_events_bed <- bed_file[bed_file[,5] %in% total_events,]
    write.table(as_events_bed,paste0("/disk1/wenqing/tmp_data/SUPPA2/ref/",type,"_as_events.bed"),quote=F,row.names=F,col.names=F,sep='\t')
    write.table(total_events_bed,paste0("/disk1/wenqing/tmp_data/SUPPA2/ref/",type,"_total_events.bed"),quote=F,row.names=F,col.names=F,sep='\t')
}

In [8]:
seq_file <- read.table("/disk1/wenqing/tmp_data/PFC_s2/data/GW26_1_1/GABA_random_100_seq.fastq",sep='\n')[,1]
seq_loc <- c()
for(i in 0:(length(seq_file)-1)){
    if(i %% 4 ==2){
        seq_loc <- c(seq_loc,i)
    }
}
all_seqs <- seq_file[seq_loc]
result_ts <- read.table('/disk1/wenqing/tmp_data/PFC_s2/transcript_mapping/random_100_seqs_ts.txt',sep='\t')
result_genome <- read.table('/disk1/wenqing/tmp_data/PFC_s2/transcript_mapping/random_100_seqs_ge.txt',sep='\t')
head(result_ts)
head(result_genome)
#write.table(all_seqs,"/disk1/wenqing/tmp_data/PFC_s2/transcript_mapping/random_100_seqs.txt",quote=F,row.names=F,col.names=F)
uniq_seq <- unique(result_genome[,5])
all_res <- data.frame()
for(one in uniq_seq){
    loc_ge <- unique(result_genome[which(result_genome[,5]==one),4])
    chr_ge <- unique(result_genome[which(result_genome[,5]==one),3])

    loc_ts <- unique(result_ts[which(result_ts[,5]==one),4])
    chr_ts <- unique(result_ts[which(result_ts[,5]==one),3])

    res_ge <- paste0(chr_ge,"_",loc_ge)
    res_ts <- paste0(chr_ts,"_",loc_ts)

    res_line <- c(one,res_ge,res_ts)
    all_res <- rbind(all_res,res_line)
}

In [15]:
####计算转录本EI和表达的相关性
iso_expr <- read.table("/disk1/wenqing/tmp_data/PFC_s2/salmon_output_untrimmed_ver/GW26_1_1/iso_wz_tpm.txt")
for(as_type in c('SE_nonAs','SE_as')){
    print(as_type)
    iso_EI <- read.table(paste0("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/EI_in_reads/GW26_1_1/EI_from_fq/",as_type,"_events_wzResNum_Over0.wzEI.txt"))
    all_ei <- c()
    all_expr <- c()
    for(i in 1:nrow(iso_EI)){
        if(!is.na(iso_EI[i,8])){
            ei_this_iso <- iso_EI[i,8]
            expr_this_iso <- iso_expr[which(iso_expr[,1]==iso_EI[i,5]),3]

            all_ei <- c(all_ei,ei_this_iso)
            all_expr <- c(all_expr,expr_this_iso)
        }
    }
    all_ei
    all_expr
    print(cor.test(all_ei,all_expr))
}

[1] "SE_nonAs"

	Pearson's product-moment correlation

data:  all_ei and all_expr
t = -1.0357, df = 38, p-value = 0.3069
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.4537731  0.1537613
sample estimates:
      cor 
-0.165684 

[1] "SE_as"

	Pearson's product-moment correlation

data:  all_ei and all_expr
t = -0.47346, df = 61, p-value = 0.6376
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.3037214  0.1901053
sample estimates:
        cor 
-0.06050984 



In [10]:
iso_EI

V1,V2,V3,V4,V5,V6,V7,V8
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>
chr1,93866284,93879206,DNTTIP2,ENST00000436063.7,protein_coding,2,6.292276e-05
chr1,109187979,109206781,ELAPOR1,ENST00000369936.2,retained_intron,12,3.195739e-03
chr1,145921556,145927484,RBM8A,ENST00000583313.7,protein_coding,200,2.715694e-03
chr1,156010221,156020950,SSR2,ENST00000368311.5,retained_intron,3,6.879156e-04
chr1,156093340,156139171,LMNA,ENST00000676385.2,protein_coding,260,1.580932e-02
chr1,156721895,156728766,ISG20L2,ENST00000368219.2,protein_coding,4,1.246145e-04
chr1,231528669,231566524,TSNAX,ENST00000366639.9,protein_coding,6,1.074691e-03
chr1,247449118,247458008,OR2B11,ENST00000641613.1,processed_transcript,6,NA
chr2,168920781,169031324,ABCB11,ENST00000650372.1,protein_coding,24,2.886350e-03
